In [1]:
# !pip install splinter
# !pip install webdriver-manager
# reference blog: https://www.codecademy.com/resources/blog/web-scraping-python-beautiful-soup-mlb-stats/

In [2]:
import requests
import os
import re
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Set up Splinter Executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [4]:
# Visit espn salaries page
url = 'http://www.espn.com/nba/salaries'
browser.visit(url)
# Search all elements with the tag div with attribute list_text, then wait 1 second before searching components
browser.is_element_present_by_css('div.list_text', wait_time=1)

False

In [5]:
# set up html parser
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
# create the dataframe
header = soup.find('tr', class_='colhead')
columns = [col.get_text() for col in header.find_all('td')]
salaries_df = pd.DataFrame(columns = columns)
salaries_df

Empty DataFrame
Columns: [RK, NAME, TEAM, SALARY]
Index: []

In [19]:
# get the players and start populating the dataframe
while True:
    players = soup.find_all('tr', attrs={'class':re.compile('row')})
    for player in players:
        # get info and salaries
        salary = [info.get_text() for info in player.find_all('td')]
        # create a temporary df
        temp_df = pd.DataFrame(salary).transpose()
        temp_df.columns = columns
        salaries_df = pd.concat([salaries_df, temp_df], ignore_index = True)
    
    # try and except to click through the pages
    try:  
        # find next button and press
        next_button =  browser.find_by_tag('div[class = "jcarousel-next"]')
        next_button.click()
    except: 
        print("End of Scrapping")
        break

End of Scrapping


In [18]:
salaries_df

RK                       NAME                    TEAM       SALARY
0    1          Stephen Curry, PG   Golden State Warriors  $45,780,966
1    2           James Harden, SG           Brooklyn Nets  $44,310,840
2    3              John Wall, PG         Houston Rockets  $44,310,840
3    4      Russell Westbrook, PG      Los Angeles Lakers  $44,211,146
4    5           Kevin Durant, PF           Brooklyn Nets  $42,018,900
5    6           LeBron James, SF      Los Angeles Lakers  $41,180,544
6    7  Giannis Antetokounmpo, PF         Milwaukee Bucks  $39,344,970
7    8         Damian Lillard, PG  Portland Trail Blazers  $39,344,900
8    9            Paul George, SG             LA Clippers  $39,344,900
9   10          Klay Thompson, SG   Golden State Warriors  $37,980,720
10  11           Kemba Walker, PG         New York Knicks  $36,016,200
11  12           Jimmy Butler, SF              Miami Heat  $36,016,200
12  13          Kawhi Leonard, SF             LA Clippers  $36,016,200
13  14          Tobias Harris, PF      Philadelphia 76ers  $35,995,950
14  15        Khris Middleton, SF         Milwaukee Bucks  $35,500,000
15  16          Anthony Davis, PF      Los Angeles Lakers  $35,361,360
16  17             Rudy Gobert, C               Utah Jazz  $35,344,828
17  18           Kyrie Irving, PG           Brooklyn Nets  $35,328,700
18  19           Bradley Beal, SG      Washington Wizards  $33,724,200
19  20          Pascal Siakam, PF         Toronto Raptors  $33,003,936
20  21            Ben Simmons, PG      Philadelphia 76ers  $33,003,936
21  22           Jrue Holiday, PG         Milwaukee Bucks  $32,413,333
22  23           Devin Booker, SG            Phoenix Suns  $31,650,600
23  24      Karl-Anthony Towns, C  Minnesota Timberwolves  $31,650,600
24  25      Kristaps Porzingis, C        Dallas Mavericks  $31,650,600
25  26             Joel Embiid, C      Philadelphia 76ers  $31,579,390
26  27         Andrew Wiggins, SF   Golden State Warriors  $31,579,390
27  28            Nikola Jokic, C          Denver Nuggets  $31,579,390
28  29             Kevin Love, PF     Cleveland Cavaliers  $31,258,256
29  30            CJ McCollum, SG  Portland Trail Blazers  $30,864,198
30  31       D'Angelo Russell, PG  Minnesota Timberwolves  $30,013,500
31  32             Chris Paul, PG            Phoenix Suns  $30,000,000
32  33         Gordon Hayward, SF       Charlotte Hornets  $29,925,000
33  34           Jamal Murray, PG          Denver Nuggets  $29,467,800
34  35         Brandon Ingram, SF    New Orleans Pelicans  $29,467,800
35  36           De'Aaron Fox, PG        Sacramento Kings  $28,103,550
36  37       Donovan Mitchell, SG               Utah Jazz  $28,103,500
37  38             Bam Adebayo, C              Miami Heat  $28,103,500
38  39           Jayson Tatum, SF          Boston Celtics  $28,103,500
39  40              Al Horford, C          Boston Celtics  $27,000,000

In [9]:
browser.quit()